In [1]:
import sys
sys.path.insert(0,'/home/noah/Desktop/large_analysis/ca_analysis/')
from matplotlib.pyplot import*
import fileread as fr
import predefined as pd
import wave_ops as wo
import time
import numpy as np
from scipy.optimize import curve_fit
import os
from scipy.constants import *
import random
# calibration=np.load('/home/noah/Desktop/large_analysis/ca_analysis/simulation_comparison/calibration.npy')
# calibration=calibration.view(np.recarray)
# def calibrate(energy_type,board,channel): 
#     bdch=int(board*8+channel) 
#     if bdch ==6: 
#         m,b=1/calibration.slope[3],calibration.offset[3]
#     elif bdch==11: 
#         m,b=1/calibration.slope[0],calibration.offset[0]
#     elif bdch==12: 
#         m,b=1/calibration.slope[1],calibration.offset[1]
#     elif bdch==35: 
#         m,b=1/calibration.slope[2],calibration.offset[2]
#     else: 
#         m,b=0,0   
#     return (energy_type-b)*m
# vec_calibrate=np.vectorize(calibrate)
path='/home/noah/Desktop/large_analysis/ca_analysis/cur_data/'
outpath='/home/noah/Desktop/large_analysis/ca_analysis/cur_data/multi_pixel/'

In [4]:
for thresh in [120,150]:
    runs=[run for run in os.listdir(path) if (run.endswith('all.dat') and run!= 'Run_0-all.dat')]
    runs.sort()
    # run=[x for x in runs if x=='Run_120-all.dat'][0]
    # data=fr.gen_output(path+run)[0]
    # data=data[pd.good_timestamps(data)]
    # data.sort(order='timestamp')
    random.shuffle(runs)
    for run in runs[0:int(len(runs)/2)]:
        multi=fr.gen_output(path+run)[0]
        multi=multi[pd.good_timestamps(multi)]
        multi.sort(order='timestamp')
        fs=35
        beg=time.time()
        energy_type='energy'
        twindow=100
        dummy='''trutharray=np.zeros(len(data))
        for i in [11,12,35]:
            bd,ch=int(i/8),int(i%8)
            trutharray=pd.lor(trutharray,\
                        pd.land(data['board']==bd,data['channel']==ch))'''

        # multi=data.copy()
        # del data
        multi[energy_type]=pd.vec_calibrate(multi[energy_type],multi['board'],multi['channel'])
        multi=multi[multi[energy_type]>thresh]
        multi=multi[multi['t0']>600]
        multi.sort(order='timestamp')
        multi=multi[pd.lor(multi['pilediff']<twindow,multi['pileup']<2)]
        # multi=multi[pd.doubles(multi,energy_type)]
        # print(time.time()-beg)

        # cut=multi.copy()

        i,j=0,0
        while i<len(multi)-1 and j<len(multi)-1:
            j=i+1
            backscattering=multi['timestamp'][j]-multi['timestamp'][i] < twindow
            energy=multi[energy_type][i]
            while backscattering and j<len(multi)-1:
                energy+=multi[energy_type][j]
                multi[energy_type][j]=-10
                j+=1
                backscattering=multi['timestamp'][j]-multi['timestamp'][i] < twindow
            multi[energy_type][i]=energy
            i=j
        print('\r'+30*' ',end='')
        print('\r'+run+' finished in {:0.2f}'.format(time.time()-beg),end='')

        np.save(file=outpath+run[:-8]+'-W-'+str(twindow)+'-T-'+str(thresh),arr=multi)       ####This is the array of interest
# np.save(file=path+'pre-multi_cut_'+str(run)+'_window-'+str(twindow),arr=cut)

Run_99-all.dat finished in 41.469